# Using *fg.py* - A Toy Example

Here, a trivial example will be presented to illustrate the proper use of *fg.py*.  In this example, the message

$\mathbf{m} = \left[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0\right]~\in~\mathbb{R}^{16}$

will be encoded into codeword $\mathbf{c}~\in~\mathbb{R}^{32}$ using an $(32, 16)$ LDPC code and transmitted over an AWGN channel 

$\mathbf{y} = \mathbf{c} + \mathbf{n}$,

where $\mathbf{n}\sim\mathcal{N}\left(\mathbf{0}, \sigma^2\mathbf{I}\right)$. Then, a message-passing decoder will be employed to recover the most likely sent codeword $\hat{\mathbf{c}}$ given observation $\mathbf{y}$. 

In [1]:
import numpy as np
import fg

### Define LDPC Factor Graph
- This LDPC code was obtained from: https://www.uni-kl.de/fileadmin/chaco/public/alists_ccsds/CCSDS_ldpc_n32_k16.alist
- *fg* requires a list of the edges between check and variable nodes. Each list within `__Check2VarEdges` corresponds to a check node and the elements of `__Check2VarEdges[i]` are the variable nodes that check `i` is connected to. 
- Messages sent between variable and check nodes are of length $2^{\mathrm{seclength}}$
- `maxdepth` parameter indicates the maximum depth of the tree induced by considering any given node as a root node

#### WARNING: The `__init()__ ` function of `class BipartiteGraph` is *hard coded* to implement a specific type of check node, i.e. `CheckNodeFFT`. The default type of check node is likely **not** what you need. Be sure to use the correct type of check node within the bipartite graph class or you will obtain some very unexpected results. 

In [2]:
class LDPC_n32_k16(fg.Encoding):
    def __init__(self, seclength=1):
        self.__Check2VarEdges = []
        self.__Check2VarEdges.append([3, 4, 6, 9, 15, 17, 24, 29])
        self.__Check2VarEdges.append([1, 4, 7, 10, 16, 18, 21, 30])
        self.__Check2VarEdges.append([1, 2, 8, 11, 13, 19, 22, 31])
        self.__Check2VarEdges.append([2, 3, 5, 12, 14, 20, 23, 32])
        self.__Check2VarEdges.append([1, 5, 6, 9, 13, 17, 21, 25])
        self.__Check2VarEdges.append([2, 6, 7, 10, 14, 18, 22, 26])
        self.__Check2VarEdges.append([3, 7, 8, 11, 15, 19, 23, 27])
        self.__Check2VarEdges.append([4, 5, 8, 12, 16, 20, 24, 28])
        self.__Check2VarEdges.append([4, 5, 9, 11, 13, 21, 26, 29])
        self.__Check2VarEdges.append([1, 6, 10, 12, 14, 22, 27, 30])
        self.__Check2VarEdges.append([2, 7, 9, 11, 15, 23, 28, 31])
        self.__Check2VarEdges.append([3, 8, 10, 12, 16, 24, 25, 32])
        self.__Check2VarEdges.append([3, 5, 9, 13, 16, 17, 25, 29])
        self.__Check2VarEdges.append([4, 6, 10, 13, 14, 18, 26, 30])
        self.__Check2VarEdges.append([1, 7, 11, 14, 15, 19, 27, 31])
        self.__Check2VarEdges.append([2, 8, 12, 15, 16, 20, 28, 32])
        super().__init__(self.__Check2VarEdges, None, seclength)
        self.maxdepth = 16

### Initialize Simulation

In [3]:
# Initialize LDPC factor graph
LDPCFactorGraph = LDPC_n32_k16()

# Create codeword
m = np.zeros(16).astype(int)

# Encode codeword
c = LDPCFactorGraph.encodemessage(m)
# NOTE: LDPCFactorGraph.encodemessage(m) will return the concatenation of belief vectors contained at each variable node. 
# This means that len(c) = 2 * len(codeword)
# For example, codeword 010 will be returned as 100110, where [1, 0] corresponds to the belief vector of v0, [0, 1]
# corresponds to the belief vector of v1, and [1, 0] corresponds to the belief vector of v2. 
# For a binary LDPC code, we can simply look at the belief that vi = 1 to recover the original codeword. 
# This is done below: 
c = c[1::2]

# We can verify that c is now of the proper length
assert len(c) == 32

# As m is the all zero message, its codeword should also be the all zero codeword. We can verify that as well: 
print(c)

Size of parity check matrix: (16, 32)
Rank of parity check matrix: 16
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 0 1 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0 0 1 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0 1 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 1 1 0 1 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1 0 1 0 0 1 1 1

### Transmit codeword over AWGN channel

In [4]:
x = 2*c-1                                       # modulate using binary phase shift keying (BPSK)
nvar = 0.001                                    # Note: sigma^2 was chosen arbitrarily
n = np.sqrt(nvar) * np.random.randn(len(c))     # generate random Gaussian noise
y = x + n                                       # generate noisy observation of c

### Decode codeword using LDPC factor graph

Here, we must convert observations $y_i$ to a vector of probabilities $\left[\mathbb{P}(y_i = 0), \mathbb{P}(y_i) = 1\right]$. To do this, we will use the formula

$\mathbb{P}(y_i = 1) = \frac{1}{1 + \exp\left(-\frac{2}{\sigma^2}y_i\right)}$




In [5]:
# Note: it is good practice to reset the factor graph if you will be using it multiple times in a row
# (e.g. averaging over multiple simulations)
LDPCFactorGraph.reset()

# Initialize variable nodes with observations
# Note that varnodeid is starts with 1 and python utilizes 0-based indexing
for varnodeid in LDPCFactorGraph.varlist:
    Pyi_1 = 1 / (1 + np.exp(-2/nvar * y[varnodeid-1]))
    LDPCFactorGraph.setobservation(varnodeid, np.array([1 - Pyi_1, Pyi_1]))
    
# Run message-passing algorithm on graph
numIterations = 1
for idxiteration in range(numIterations):
    LDPCFactorGraph.updatechecks()
    LDPCFactorGraph.updatevars()
    
# Extract information from graph
cHt = np.zeros(32)

for varnodeid in LDPCFactorGraph.varlist:
    tmp = LDPCFactorGraph.getextrinsicestimate(varnodeid)
    cHt[varnodeid - 1] = np.argmax(tmp)

# Print out true codeword and codeword codeword estimate
print('True codeword: \n' + str(c))
print('Estimated codeword: \n' + str(cHt))
print('Error rate: ' + str(np.sum(c != cHt) / 32))

True codeword: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Estimated codeword: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Error rate: 0.0


C:\Users\jamis\AppData\Local\Temp/ipykernel_21480/1855023051.py:8: RuntimeWarning: overflow encountered in exp
  Pyi_1 = 1 / (1 + np.exp(-2/nvar * y[varnodeid-1]))
